<a href="https://colab.research.google.com/github/Rogerio-mack/work/blob/main/Helaine_Julia_CNIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd

!pip install pdflib
!pip install db-sqlite3

from pdflib import Document

from google.colab import drive
drive.mount('/content/drive')

doc = Document("/content/drive/MyDrive/MackPrevIA/Exemplos de CNIS/CNIS-Cristina Mitiyo Maru Inagaki.pdf")
page1 = doc.get_page(1).lines
#print(page1)

client = {
    "Nome": page1[5].split("Nome: ", 1)[1],
    "NIT": page1[5].split("NIT: ", 1)[1][:14],
    "CPF": page1[5].split("CPF: ", 1)[1][:14],
    "Nome da mãe": page1[6].split("Nome da mãe: ", 1)[1],
    "Data de nascimento": page1[6].split("Data de nascimento: ", 1)[1][:10],
}

client = pd.DataFrame.from_records(client, index=[0])
client["Data de criação do arquivo"] = page1[3].split("Extrato Previdenciário")[1].strip()

client.head()

def check_empty_values(val: list) -> list:
    for i in range(5, len(val)):

        # Data Fim
        if i == 5 and "/" not in val[i]:
            val.insert(i, None)

        # Últ. Remun.
        elif i == 7:
            if "/" not in val[i]:
                val.insert(i, None)

    # Indicadores e/ou Últ. Remun. se for preciso
    while len(val) < 9:
        val.append(None)

    val.append([])
    return val

# Substitui espaços em brancos maiores que 1 pelo caracter &
def remove_blank_spaces(line: str) -> str:
    return re.sub('  +', '&', line)

# Retorna o index da coluna dada a palavra
def get_colum_index(page: list, var: str) -> int:
    for line in page:
        if var in line:
            return page.index(line)

    return -1

def check_empty_values_rem(val: list) -> list:

    # For loop com entre os valores das remunerações
    for index in range(0, 9, 3):

        if index < len(val):  # Verificando a existência dos dados

            if val[index][2] == '/':  # Verificando a existência da data

                if val[index + 1][-3] == ',':  # Verificando a existência da remuneração

                    if index + 2 < len(val):  # Verificando a existência dos dados

                        if val[index + 2][
                                2] == '/':  # Verificando a existência dos indicadores - LBYL
                            val.insert(index + 2, None)

                    else:  # Inserindo o valor vazio caso os dados sejam inexistentes
                        val.append(None)

        else:
            for i in range(3):
                val.append(None)

    return val  # Retornando a lista

# Função que normaliza o json
def rem_list_to_json_normalized(indexes: list, val: list, seq: str) -> list:
    rem_dict = {}
    rem_list = []

    for i in range(0, 9, 3):
        if val[i]:
            rem_dict = {
                index: value
                for index, value in zip(indexes[:3], val[i:i + 3])
            }
            rem_dict["Seq."] = seq
            rem_list.append(rem_dict)

    return rem_list

# Filtrando Relações Previdenciárias e removendo colunas
rel_prev = [line for line in page1 if line[0].isnumeric()]

# Pegando colunas Padrões das Relações
colunas = page1[get_colum_index(page1, "Seq.")]
colunas = remove_blank_spaces(colunas)
# colunas = colunas[:len(colunas) - 12] + "&Indicadores&Remuneracoes"
colunas = colunas[:len(colunas) - 12] + "&Indicadores"  # Versao normalizada
indexes_rel = colunas.split("&")

# Pegando colunas Padrões das Remunerações
colunas = page1[get_colum_index(page1, "Competência")]
colunas = remove_blank_spaces(colunas)
indexes_rem = colunas.split("&")

ult_rel: int = 0
seq: int
rem_final: list = []
df_relacao: list = []
df_remuneracao: list = []

# Função para normalizar as remunerações
def clean_up_data_normalized(dados: list, linha: list) -> None:
    global seq, rem_final

    line = remove_blank_spaces(linha).split("&")  # Pegando a linha

    if line[0].isnumeric(
    ):  # Verificando se o primeiro valor da linha é numérico
        values: list = check_empty_values(line)
        seq = line[0]

        result: dict = {index: value for index, value in zip(indexes_rel, values)}
        #print(result)
        df_relacao.append(pd.DataFrame.from_records(result, index=[0]))
        #df_relacao[len(df_relacao)] = 'CPF'
        #df_relacao.append(client.get('CPF'))
        # Acrescentado por Ana, Julia, Helaine
        

    else:
        rem: list = check_empty_values_rem(line)
        rem_json: list = rem_list_to_json_normalized(indexes_rem, rem, seq)

        dfs: list = [pd.DataFrame.from_records(elem, index=[0]) for elem in rem_json]
        
        df_remuneracao.append(pd.concat(dfs, ignore_index=True))

# Percorrendo lista de relações
for n in range(len(rel_prev)):
    clean_up_data_normalized(client, rel_prev[n])

# Alterando as opções do print do Dataframe
pd.set_option("display.max_rows", None, "display.max_columns", None)

#print(df_relacao)
df_relacao = pd.concat(df_relacao, ignore_index=True)
df_relacao.drop_duplicates()
#df_relacao.insert(len(df_relacao), 'CPF', client.get('CPF'))
df_relacao.insert(0,  "CPF", client["CPF"])
df_relacao["CPF"] = df_relacao["CPF"].ffill()
#print(df_relacao.head())
#print("\n\n\n\n")
df_remuneracao = pd.concat(df_remuneracao, ignore_index=True)
#print(df_remuneracao)

def clean_data(seq, linha, df_remuneracao):

  line = remove_blank_spaces(linha).split("&")

  if line[0].isnumeric():
      seq = line[0]

  else:

    rem: list = check_empty_values_rem(line)
    rem_json: list = rem_list_to_json_normalized(indexes_rem, rem, seq)

    dfs: list = [pd.DataFrame.from_records(elem, index=[0]) for elem in rem_json]
    
    df_remuneracao.append(pd.concat(dfs, ignore_index=True))  

  return seq



def get_rem(page: list) -> pd.DataFrame:

    rel_prev = [line for line in page if line[0].isnumeric()]

    colunas = page[get_colum_index(page, "Competência")]
    colunas = remove_blank_spaces(colunas)
    indexes_rem = colunas.split("&")
    df_remuneracao: list = []
    seq = 0

    for n in range(len(rel_prev)):
      seq = clean_data(seq, rel_prev[n], df_remuneracao)

    df_remuneracao = pd.concat(df_remuneracao, ignore_index=True)

    return df_remuneracao


df_remuneracao = get_rem(page1)


#print(df_remuneracao.head(10))


def main():
  print("DADOS DO BENEFICIÁRIO")
  print("Nome do Beneficiário: ", client.get("Nome").values[0])
  print("NIT: ", client.get("NIT").values[0])
  print("CPF: ", client.get("CPF").values[0])
  print("Nome da mãe: ", client.get("Nome da mãe").values[0])
  print("Data de nascimento: ", client.get("Data de nascimento").values[0])
  print("\n\n\n")
  print("RELAÇÕES")
  print(df_relacao)

main()
  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DADOS DO BENEFICIÁRIO
Nome do Beneficiário:  CRISTINA MITIYO MARU INAGAKI
NIT:  122.05604.68-8
CPF:  088.153.748-92
Nome da mãe:  ALICE AKIY KAWAMATA
Data de nascimento:  03/01/1965




RELAÇÕES
              CPF         Código Emp.    Data Fim          Data Início  \
0  088.153.748-92          47.192.091  28/06/1985           13/05/1985   
1  088.153.748-92  56.991.888/0008-00  12/07/1991           08/09/1987   
2  088.153.748-92  61.192.571/0002-40  15/07/1991   JOHNSON COMERCIO E   

  Indicadores             NIT                      Origem do Vínculo Seq.  \
0    AVRC-DEF  122.05604.68-8      COTIA TRABALHO TEMPORARIO LTDA EM    1   
1     ACNISVR  122.05604.68-8  ENGESA ENGENHEIROS ESPECIALIZADOS S A    2   
2   Empregado  122.05604.68-8                               JOHNSON     3   

  Tipo Filiado no Vínculo Últ. Remun.  
0               Empregado     

In [ ]:
df_relacao.head()